#Chatbot RNN + TCN +Attention

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Tue Apr 11 10:57:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##Imports

In [ ]:
import torch
import torch.nn as nn
from torch import optim
import numpy as np
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools
from torch.utils.data import Dataset,DataLoader
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from collections import Counter
# from torch.nn.modules.conv import TemporalConvNet

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
CUDA= torch.cuda.is_available()
device=torch.device("cuda" if CUDA else "cpu")

## Load Hindi Data

In [ ]:
base='/content/drive/MyDrive/Chatbot_research'
os.chdir(base)
##Dialog File Train
corpus_name='daily_dialog_corpus_hindi'
train_file=os.path.join(base,'formatted_hindi_train_merged.txt')
test_file=os.path.join(base,'formatted_hindi_test.txt')
val_file=os.path.join(base,'formatted_hindi_val.txt')

In [ ]:
print('\n Sample Train Lines')
with open(train_file,'r') as f:
  lines=f.readlines()

for line in lines[35:40]:
  print(line)


 Sample Train Lines
यह फर्जी है। मुझे काम पर कोई तनाव महसूस नहीं होता है, और मेरा प्रेम जीवन व्यावहारिक रूप से अस्तित्वहीन है। यह राशि की सारी बातें बकवास हैं।	नहीं, ऐसा नहीं है, आपका ज्योतिष संकेत आपको आपके व्यक्तित्व के बारे में बहुत कुछ बता सकता है। देखिए? इसमें कहा गया है कि मेष ऊर्जावान होता है और उसे सामाजिक जीवन पसंद होता है।

नहीं, ऐसा नहीं है, आपका ज्योतिष संकेत आपको आपके व्यक्तित्व के बारे में बहुत कुछ बता सकता है। देखिए? इसमें कहा गया है कि मेष ऊर्जावान होता है और उसे सामाजिक जीवन पसंद होता है।	खैर, आप निश्चित रूप से उन मानदंडों से मेल खाते हैं, लेकिन वे इतने व्यापक हैं कि वे किसी पर भी लागू हो सकते हैं। यह मेरे बारे में क्या कहता है?

खैर, आप निश्चित रूप से उन मानदंडों से मेल खाते हैं, लेकिन वे इतने व्यापक हैं कि वे किसी पर भी लागू हो सकते हैं। यह मेरे बारे में क्या कहता है?	एक मकर राशि गंभीर और व्यावहारिक होती है। उसे पारंपरिक तरीकों से काम करना पसंद है। यह बिल्कुल आपकी तरह लगता है।

फ्रैंक की शादी हो रही है, क्या आप इस पर विश्वास करते हैं?	क्या वह वास्तव में है?

क्या वह

In [ ]:
print('\n Sample test Lines')
with open(test_file,'r') as f:
  lines=f.readlines()
print(len(lines))
for line in lines[35:40]:
  print(line)


 Sample test Lines
6740
क्या आप एक नेता हैं या अनुयायी?	मैं लोगों का नेतृत्व करने की कोशिश नहीं करता। मैं इसके बजाय सभी के साथ सहयोग करूंगा और मिलकर काम करके काम पूरा करूंगा।

मैं लोगों का नेतृत्व करने की कोशिश नहीं करता। मैं इसके बजाय सभी के साथ सहयोग करूंगा और मिलकर काम करके काम पूरा करूंगा।	क्या आप सोचते हैं कि आप अपने आपको अंग्रेजी में आसानी से समझ सकते हैं?

क्या आप सोचते हैं कि आप अपने आपको अंग्रेजी में आसानी से समझ सकते हैं?	हां, ज्यादातर परिस्थितियों में।

हां, ज्यादातर परिस्थितियों में।	क्या आप यात्रा के लिए उपलब्ध हैं?

क्या आप यात्रा के लिए उपलब्ध हैं?	हां, मुझे यात्रा करना पसंद है। मैं युवा और अविवाहित हूं। मुझे अक्सर यात्रा करने में कोई समस्या नहीं है।



## Load And Trim Data

In [ ]:
PAD_tok=0
SOS_tok=1
EOS_tok=2
OOV_tok=3

class Vocabulary:
  def __init__(self,name):
    self.name=name
    self.word2idx={}
    self.word2count={}
    self.idx2word={PAD_tok:'PAD',SOS_tok:'SOS',EOS_tok:'EOS',OOV_tok:'OOV'}
    self.num_words=4
    self.word2idx['OOV']=3
  
  def addLine(self,line):
    for word in line.split(' '):
      self.addWord(word)
  def addWord(self,word):
    if word not in self.word2idx:
      self.word2idx[word]=self.num_words
      self.word2count[word]=1
      self.idx2word[self.num_words]=word
      self.num_words+=1
    else:
      self.word2count[word] += 1

In [ ]:
def normalizeString(text):
  # s = unicodeToASCII(s.lower().strip())
   text = text.lower()
   text = re.sub('((www.[^s]+)|(https?://[^s]+))','',text)
   text = re.sub('@[^s]+','',text)
   text = re.sub('[s]+', ' ', text)
   text = re.sub(r'#([^s]+)', r'1', text)
   text = re.sub('[.!:?-]', '', text)
   text = re.sub('[a-zA-Z0-9]','',text)
   text = re.sub(' +', ' ',text)
   text = text.strip('""')
   return text

In [ ]:
print('Reading File please wait...')
lines=open(train_file,).read().strip().split('\n')
pairs_data_train=[[normalizeString(s) for s in pair.split('\t')] for pair in lines]
print('Done Reading.....')

Reading File please wait...
Done Reading.....


In [ ]:
print('Reading File please wait...')
lines=open(val_file,).read().strip().split('\n')
pairs_data_val=[[normalizeString(s) for s in pair.split('\t')] for pair in lines]
print('Done Reading.....')

Reading File please wait...
Done Reading.....


In [ ]:
MAX_LENGTH=16
# Load/Assemble voc and pairs
save_dir = os.path.join("data", "save")
def filterPair(p):
  return len(p[0].split())<MAX_LENGTH and len(p[1].split())<MAX_LENGTH

def filterPairs(pairs):
  return [pair for pair in pairs if filterPair(pair)]

pairs_train=pairs_data_train
pairs_train=[pair for pair in pairs_train if len(pair)>1]
print("There are {} pairs in the  train dataset".format(len(pairs_train)))
pairs_train=filterPairs(pairs_train)
print('After filtering, there are {} pair'.format(len(pairs_train)))


pairs_val=pairs_data_val
pairs_val=[pair for pair in pairs_val if len(pair)>1]
print("There are {} pairs in the validation dataset".format(len(pairs_val)))
pairs_val=filterPairs(pairs_val)
print('After filtering, there are {} pair'.format(len(pairs_val)))

There are 76053 pairs in the  train dataset
After filtering, there are 43534 pair
There are 7069 pairs in the validation dataset
After filtering, there are 4076 pair


##Creating Vocabulary

In [ ]:
print('Creating Vocabulary....')
vocab=Vocabulary(corpus_name)

for pair in pairs_train:
  vocab.addLine(pair[0])
  vocab.addLine(pair[1])
print("counted words:",vocab.num_words)

Creating Vocabulary....
counted words: 13906


##Pretrained Embeddings

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git

Cloning into 'fastText'...
remote: Enumerating objects: 3930, done.
remote: Counting objects: 100% (958/958), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 3930 (delta 867), reused 817 (delta 817), pack-reused 2972
Receiving objects: 100% (3930/3930), 8.24 MiB | 17.26 MiB/s, done.
Resolving deltas: 100% (2505/2505), done.
Updating files: 100% (526/526), done.


In [ ]:
%cd fastText

/content/drive/MyDrive/Chatbot_research/fastText


In [ ]:
!pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/drive/MyDrive/Chatbot_research/fastText
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp39-cp39-linux_x86_64.whl size=4381862 sha256=f9b3049822451a467ff6721ba94446c9b391f50906967f23ef55347c60d461e0
  Stored in directory: /tmp/pip-ephem-wheel-cache-iv7zym6m/wheels/73/fc/58/7576e77ca4efea93f598beb2abdd885074dacc456b5f3e318d
Successfully built fasttext


In [ ]:
%cd ..

/content/drive/MyDrive/Chatbot_research


In [ ]:
import fasttext
ft_model = fasttext.load_model('cc.hi.300.bin')

In [ ]:
emb_dim=300
embedding_matrix = np.zeros((vocab.num_words, emb_dim))

for i, word in enumerate(vocab.word2idx):
    if word in ft_model:
        embedding_matrix[i] = ft_model[word]
    else:
        embedding_matrix[i] = np.random.randn(emb_dim)

In [ ]:
embedding_layer = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_matrix))
embedding_layer.requires_grad = False

##Prepare the Data

In [ ]:
def indexesFromSentence(vocab, sentence):
    return [vocab.word2idx[word] if word in vocab.word2idx.keys() else vocab.word2idx['OOV'] for word in sentence.split(' ')] + [EOS_tok]


def zeroPadding(l, fillvalue=PAD_tok):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_tok):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_tok:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len

##Model

###1.Encoder

In [ ]:
class EncoderRNN(nn.Module):
  def __init__(self,hidden_size,embedding,n_layers=1,dropout=0):
    super(EncoderRNN,self).__init__()
    self.n_layer=n_layers
    self.hidden_size=hidden_size
    self.embedding=embedding
    self.gru=nn.GRU(hidden_size,hidden_size,n_layers,dropout=dropout,bidirectional=True)

  def forward(self,input_seq,input_length,hidden=None):
    embedded=self.embedding(input_seq)
    packed=torch.nn.utils.rnn.pack_padded_sequence(embedded,input_length)
    output,hidden=self.gru(packed,hidden)
    output,_=torch.nn.utils.rnn.pad_packed_sequence(output)
    output=output[:,:,:self.hidden_size]+output[:,:,self.hidden_size:]
    return output,hidden

In [ ]:
import torch.nn as nn
from torch.nn.utils import weight_norm


class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = num_channels
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_inputs*2
            out_channels = num_inputs*2
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [ ]:
class EncoderTCN(nn.Module):
    def __init__(self, hidden_size,embedding, n_layers, kernel_size=3, dropout=0.2):
        super(EncoderTCN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding
        self.tcn = TemporalConvNet(hidden_size,n_layers,kernel_size, dropout=dropout)
        self.fc = nn.Linear(hidden_size*2, hidden_size)

    def forward(self, input_seq, input_length):
        embedded = self.embedding(input_seq)
        output = self.tcn(embedded.permute(0,2,1))
        output = output.permute(0,2,1)
        output=self.fc(output)
        return output

###2.Decoder

In [ ]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [ ]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # attn_weights_tcn=self.attn(rnn_output, encoderTCN_outputs)
        # attn_weights=(0.7*(attn_weights_rnn))+(0.3*(attn_weights_tcn))
        # attn_weights=torch.cat((attn_weights_rnn,attn_weights_tcn),dim=1)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        # context=context.view(-1, context.shape[-1]*2)
        # print('context:',context.shape)
        # print('rnn_output: ',rnn_output.shape)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

## Model Training

In [ ]:
def maskNLLLoss(decoder_out,target,mask):
  nTotal=mask.sum()
  target=target.view(-1,1)
  gathered_tensor=torch.gather(decoder_out,1,target)
  crossEntropy= -torch.log(gathered_tensor)
  loss=crossEntropy.masked_select(mask)
  loss=loss.mean()
  loss=loss.to(device)
  return loss,nTotal.item()

In [ ]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoderRNN, decoder, embedding,
          encoderRNN_optimizer, decoder_optimizer, batch_size, clip,encoderTCN,encoderTCN_optimizer,beta):

    encoderRNN.train()
    encoderTCN.train()
    decoder.train()
    # Zero gradients
    encoderRNN_optimizer.zero_grad()
    encoderTCN_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # Lengths for rnn packing should always be on the cpu
    lengths = lengths.to("cpu")

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoderRNN_outputs, encoder_hidden = encoderRNN(input_variable, lengths)
  
    encoderTCN_outputs=encoderTCN(input_variable, lengths)
   
    encoder_outputs=((beta)*encoderRNN_outputs)+((1-beta)*encoderTCN_outputs)

  
    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_tok for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    # use_teacher_forcing = False
    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoderRNN.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(encoderTCN.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoderRNN_optimizer.step()
    encoderTCN_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [ ]:
def evaluateInput(input_variable, lengths, target_variable, mask, max_target_len, encoderRNN, decoder, 
                  embedding,encoderRNN_optimizer, decoder_optimizer, batch_size, clip,encoderTCN,encoderTCN_optimizer,beta):

  with torch.no_grad():
    encoderRNN.eval()
    encoderTCN.eval()
    decoder.eval()
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
      # Lengths for rnn packing should always be on the cpu
    lengths = lengths.to("cpu")

      # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    encoderRNN_outputs, encoder_hidden = encoderRNN(input_variable, lengths)
    encoderTCN_outputs=encoderTCN(input_variable, lengths)
    encoder_outputs=((beta)*encoderRNN_outputs)+((1-beta)*encoderTCN_outputs)

      # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_tok for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    for t in range(max_target_len):
      decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
      _, topi = decoder_output.topk(1)
      decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
      decoder_input = decoder_input.to(device)
      # Calculate and accumulate loss
      mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
      loss += mask_loss
      print_losses.append(mask_loss.item() * nTotal)
      n_totals += nTotal

  return sum(print_losses) / n_totals


In [ ]:
def trainIters(model_name, voc, pairs_train,pairs_val, encoderRNN, decoder, encoderRNN_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, 
               save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename,encoderTCN,encoderTCN_optimizer,epoch,beta):
    num_batch=len(pairs_train)//batch_size + int(len(pairs_train)% batch_size != 0)
    n_iteration=epoch*num_batch
    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs_train) for _ in range(batch_size)])
                      for _ in range(n_iteration)]
    validation_batches=[batch2TrainData(voc, [random.choice(pairs_val) for _ in range(batch_size)])
                      for _ in range(n_iteration)]
    
    training_loss=[]
    validation_loss=[]
    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_train_loss = 0
    print_val_loss = 0
    epoch=1
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        validation_batch = validation_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch
        # Run a training iteration with batch
        train_loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoderRNN,
                     decoder, embedding, encoderRNN_optimizer, decoder_optimizer, batch_size, clip,encoderTCN,encoderTCN_optimizer,beta)
        
        input_variable, lengths, target_variable, mask, max_target_len = validation_batch
        val_loss = evaluateInput(input_variable, lengths, target_variable, mask, max_target_len, encoderRNN,
                     decoder, embedding, encoderRNN_optimizer, decoder_optimizer, batch_size, clip,encoderTCN,encoderTCN_optimizer,beta)
        print_train_loss += train_loss
        print_val_loss+= val_loss
        # Print progress
        if iteration % num_batch == 0:
            print_loss_avg_train = print_train_loss / num_batch
            print_loss_avg_val = print_val_loss / num_batch
            training_loss.append(print_loss_avg_train)
            validation_loss.append(print_loss_avg_val)
            print("Epoch: {}; Percent complete: {:.1f}%; Average train loss: {:.4f}   Average val loss: {:.4f}".format(epoch, iteration / n_iteration * 100, print_loss_avg_train,print_loss_avg_val))
            print_train_loss = 0
            print_val_loss= 0
            epoch+=1
        if epoch==5:
          embedding.requires_grad=True
    
    return training_loss,validation_loss     

###Initialize the model

In [ ]:
# Configure models
model_name = 'cb_model_rnn_attention'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 300
encoder_n_layers = 4
decoder_n_layers = 2
dropout = 0.2
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
# checkpoint_iter = 10000
# loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    vocab.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
# embedding = nn.Embedding(vocab.num_words, hidden_size)
# if loadFilename:
#     embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoderRNN = EncoderRNN(hidden_size, embedding_layer, encoder_n_layers, dropout)
encoderTCN = EncoderTCN(hidden_size, embedding_layer, n_layers=encoder_n_layers)
decoder = LuongAttnDecoderRNN(attn_model, embedding_layer, hidden_size, vocab.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoderRNN.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoderRNN = encoderRNN.to(device)
encoderTCN=encoderTCN.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [ ]:
# Configure training/optimization
clip = 50
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 0
print_every = 1
save_every = 500
epoch=40

# Ensure dropout layers are in train mode
encoderRNN.train()
encoderTCN.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoderRNN_optimizer = optim.Adam(encoderRNN.parameters(), lr=learning_rate)
encoderTCN_optimizer = optim.Adam(encoderTCN.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
# encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate,momentum=0.9)
# decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate * decoder_learning_ratio,momentum=0.09)
# if loadFilename:
#     encoderRNN_optimizer.load_state_dict(encoder_optimizer_sd)
#     decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# # If you have cuda, configure cuda to call
# for state in encoder_optimizer.state.values():
#     for k, v in state.items():
#         if isinstance(v, torch.Tensor):
#             state[k] = v.cuda()

# for state in decoder_optimizer.state.values():
#     for k, v in state.items():
#         if isinstance(v, torch.Tensor):
#             state[k] = v.cuda()

# Run training iterations
print("Starting Training!")
train_loss,val_loss=trainIters(model_name, vocab, pairs_train,pairs_val, encoderRNN, decoder, encoderRNN_optimizer, decoder_optimizer,
           embedding_layer, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename,encoderTCN,encoderTCN_optimizer,epoch,beta=1)

##Evaluate

In [ ]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder,encoderTCN, decoder,beta):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.encoderTCN=encoderTCN
        self.decoder = decoder
        self.beta=beta

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoderRNN_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        encoderTCN_outputs=encoderTCN(input_seq, input_length)
        encoder_outputs=((self.beta)*encoderRNN_outputs)+((1-self.beta)*encoderTCN_outputs)
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_tok
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [ ]:
def evaluate(searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to("cpu")
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.idx2word[token.item()] for token in tokens]
    return decoded_words

##Chat

In [ ]:
def evalInput(searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate( searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [ ]:
# Set dropout layers to eval mode
encoderTCN.eval()
decoder.eval()
encoderRNN.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoderRNN,encoderTCN, decoder,beta=1)
# searcher = BeamSearchDecoder(encoderRNN,encoderTCN, decoder,0.7,3)

# Begin chatting (uncomment and run the following line to begin)
evalInput(searcher, vocab)

> नमस्कार कैसे हो तुम?
Bot: धन्यवाद।
> चलो कहीं बाहर खाने चलते हैं
Bot: ठीक है।
> मैं आज बहुत खुश हूं
Bot: क्या आप जानते हैं कि आपको किस तरह का कमरा चाहिए


KeyboardInterrupt: ignored

###Evaluation Metrics

In [ ]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rouge import Rouge 
def calculateRogue(test_data):
  prediction=[]
  actual=[]
  for pair in test_data:
    pred = evaluate(searcher, vocab, pair[0])
    pred[:] = [x for x in pred if not (x == 'EOS' or x == 'PAD')]
    pred=' '.join(pred)
    if len(pred)<=0:
      continue
    prediction.append(pred)
    actual.append(pair[1])
  rouge = Rouge()
  scores = rouge.get_scores(prediction, actual,avg=True)
  print(scores)
  return prediction,actual

In [ ]:
##Bleu Score
def bleu_score(guess, answer):
    """Compute approximate BLEU score between guess and a set of answers."""
    bleu1=sentence_bleu([normalizeString(answer).split()],normalizeString(guess).split(" "),weights=[1,0,0,0])
    bleu2=sentence_bleu([normalizeString(answer).split()],normalizeString(guess).split(" "),weights=[0.5,0.5,0,0])
    bleu3=sentence_bleu([normalizeString(answer).split()],normalizeString(guess).split(" "),weights=[0.3,0.3,0.3,0])
    return [bleu1,bleu2,bleu3]

In [ ]:
##f1_score
def prec_recall_f1_score(pred_items, gold_items)->float:
    common = Counter(gold_items) & Counter(pred_items)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_items)
    recall = 1.0 * num_same / len(gold_items)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def f1_score(guess, answer)-> float:
    """Return the max F1 score between the guess and *any* answer."""
    if guess is None or answer is None:
        return 0
    g_tokens = normalizeString(guess).split()
    a_token=normalizeString(answer).split()
    return prec_recall_f1_score(g_tokens,a_token )

In [ ]:
##Meteor Score
def _meteor_score(guess,answer):
  return meteor_score([normalizeString(answer).split()],normalizeString(guess).split())

In [ ]:
def evaluateMetrics(test_data):
  num_samples=len(test_data)
  f1=0.0
  bleu=[0.0,0.0,0.0]
  meteor=0.0
  for pair in test_data:
    pred = evaluate(searcher, vocab, pair[0])
    pred[:] = [x for x in pred if not (x == 'EOS' or x == 'PAD')]
    pred=' '.join(pred)
    actual=pair[1]
    f1+=f1_score(pred,actual)
    bleu=[a+b for a,b in zip(bleu,bleu_score(pred,actual))]
    meteor+=_meteor_score(pred,actual)
  f1=(f1/num_samples)*100
  bleu[:]=[(x/num_samples)*100 for x in bleu]
  meteor=(meteor/num_samples)*100
  print(f'F1_score: {f1:.2f}')
  print(f'BLEU-1_score: {bleu[0]:.3f}')
  print(f'BLEU-2_score: {bleu[1]:.3f}')
  print(f'BLEU-3_score: {bleu[2]:.2f}')
  print(f'meteor_score: {meteor:.2f}')
  return f1,bleu,meteor

In [ ]:
print('Reading lines from formatted test files please wait...')
test_lines=open(test_file,encoding='utf-8').read().strip().split('\n')
pairs_test_data=[[normalizeString(s) for s in pair.split('\t')] for pair in test_lines]
print('Done Reading..... total train length {}'.format(len(pairs_test_data)))
pairs_test_data=filterPairs(pairs_test_data)
print('After filtering, there are {} pair'.format(len(pairs_test_data)))

In [ ]:
f1,bleu,meteor=evaluateMetrics(pairs_test_data)

In [ ]:
pred,actual=calculateRogue(pairs_test_data)